In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline


loan = pd.read_csv('accepted_2007_to_2018Q4.csv')

return_number = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
}

def convert_date(d):
    try:
        return datetime.date(year=int(d[4:]), month=return_number[d[:3]], day=1)
    except:
        pass

loan['issue_d'] = loan['issue_d'].apply(convert_date)
loan['earliest_cr_line'] = loan['earliest_cr_line'].apply(convert_date)

loan = loan[loan.issue_d < datetime.date(2015,7,1)]
loan = loan[loan.issue_d >= datetime.date(2010,1,1)]

loan = loan[loan.term == ' 36 months']

loan['earliest_cr_line'] = loan.apply(lambda x: (x['issue_d'] - x['earliest_cr_line']).days, axis=1)

loan = loan[(loan.loan_status == 'Fully Paid') | (loan.loan_status == 'Charged Off')]

loan = loan[loan.annual_inc < 1000000]

loan['ln_annual_inc'] = np.log(loan.annual_inc)

loan = loan[loan.revol_util < 150]
loan['ln_revol_bal'] = np.log(loan.revol_bal+1)

loan['ln_earliest_cr_line'] = np.log(loan.earliest_cr_line)

loan['ln_open_acc'] = np.log(loan.open_acc)

loan.rename(columns = {'delinq_2yrs': 'num_delinq_2yrs'}, inplace=True)
loan['delinq_2yrs'] = (loan['num_delinq_2yrs'] >= 1)

loan.rename(columns = {'pub_rec': 'num_pub_rec'}, inplace=True)
loan['pub_rec'] = (loan['num_pub_rec'] >= 1)

loan.rename(columns = {'inq_last_6mths': 'num_inq_last_6mths'}, inplace=True)
loan['inq_last_6mths'] = (loan['num_inq_last_6mths'] >= 1)

loan['target'] = (loan['loan_status'] == 'Fully Paid')

In [2]:
col_list = ['loan_amnt', 'int_rate', 'ln_annual_inc', 'dti', 'fico_range_high', 'num_delinq_2yrs', 'ln_earliest_cr_line', 'num_inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'ln_open_acc', 'num_pub_rec', 'ln_revol_bal', 'revol_util', 'total_acc']


In [17]:
# We will use 2010-2013 to tune our models, and save data from 2014 and the first 
# 6 months of 2015 for walk-forward testing

train = loan[loan.issue_d < datetime.date(2014, 1, 1)]

In [6]:
loan.shape

(449486, 158)

In [18]:
train.shape

(166313, 159)

In [16]:
print('2010: {}'.format(sum([x.year == 2010 for x in train.issue_d])))
print('2011: {}'.format(sum([x.year == 2011 for x in train.issue_d])))
print('2012: {}'.format(sum([x.year == 2012 for x in train.issue_d])))
print('2013: {}'.format(sum([x.year == 2013 for x in train.issue_d])))

2010: 8445
2011: 14092
2012: 43423
2013: 100353


In [47]:
import statsmodels.api as sm

y = train['target']
X1 = train[['loan_amnt', 'ln_annual_inc', 'dti', 'fico_range_high', 'delinq_2yrs', 'num_delinq_2yrs', 'ln_earliest_cr_line', 'inq_last_6mths', 'num_inq_last_6mths', 'ln_open_acc', 'pub_rec', 'num_pub_rec', 'ln_revol_bal', 'revol_util', 'total_acc']]
X2 = pd.concat([X1, train.int_rate, pd.get_dummies(train.grade)], axis=1)
X3 = pd.concat([X1, train.int_rate, pd.get_dummies(train.sub_grade)], axis=1)

In [32]:
type(y)

pandas.core.series.Series

In [33]:
type(X1)

pandas.core.frame.DataFrame

In [36]:
X1.shape

(166313, 17)

In [56]:
logit = sm.Probit(y, X1.astype(float)).fit()
logit_me = logit.get_margeff()
logit_me.summary()

Optimization terminated successfully.
         Current function value: 0.363638
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                 target
Method:                          dydx
At:                           overall
=======================================================================================
                         dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
loan_amnt           -6.524e-07   1.26e-07     -5.195      0.000   -8.99e-07   -4.06e-07
ln_annual_inc           0.0235      0.002     13.166      0.000       0.020       0.027
dti                    -0.0023      0.000    -18.336      0.000      -0.002      -0.002
fico_range_high         0.0003   2.65e-05      9.721      0.000       0.000       0.000
delinq_2yrs            -0.0009      0.004     -0.261      0.794      -0.008       0.006
num_delinq_2yrs        -0.0030      0.002     -1.635      0.102      -0.007       0.001
ln_earliest_cr_line    -0.0197      0.002    -10.648      0.000      -0.023      -0.016
inq_last_6mths         -0.0117      0.003     -4.510      0.000      -0.017      -0.007
num_inq_last_6mths     -0.0170      0.001    -13.759      0.000      -0.019      -0.015
ln_open_acc            -0.0399      0.003    -15.119      0.000      -0.045      -0.035
pub_rec                -0.0017      0.005     -0.345      0.730      -0.011       0.008
num_pub_rec            -0.0003      0.004     -0.074      0.941      -0.007       0.007
ln_revol_bal            0.0138      0.001     15.499      0.000       0.012       0.016
revol_util             -0.0010    4.6e-05    -22.138      0.000      -0.001      -0.001
total_acc               0.0021      0.000     20.139      0.000       0.002       0.002
=======================================================================================
"""

In [57]:
logit = sm.Probit(y, X2.astype(float)).fit()
logit_me = logit.get_margeff()
logit_me.summary()

Optimization terminated successfully.
         Current function value: 0.356357
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                 target
Method:                          dydx
At:                           overall
=======================================================================================
                         dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
loan_amnt           -1.761e-06   1.34e-07    -13.134      0.000   -2.02e-06    -1.5e-06
ln_annual_inc           0.0568      0.002     27.824      0.000       0.053       0.061
dti                    -0.0012      0.000     -9.379      0.000      -0.001      -0.001
fico_range_high         0.0004   4.44e-05      9.824      0.000       0.000       0.001
delinq_2yrs             0.0079      0.004      2.199      0.028       0.001       0.015
num_delinq_2yrs        -0.0010      0.002     -0.534      0.593      -0.005       0.003
ln_earliest_cr_line    -0.0032      0.002     -1.689      0.091      -0.007       0.001
inq_last_6mths          0.0015      0.003      0.593      0.553      -0.004       0.007
num_inq_last_6mths     -0.0123      0.001     -9.929      0.000      -0.015      -0.010
ln_open_acc            -0.0201      0.003     -7.623      0.000      -0.025      -0.015
pub_rec                 0.0090      0.005      1.798      0.072      -0.001       0.019
num_pub_rec             0.0015      0.004      0.414      0.679      -0.006       0.009
ln_revol_bal            0.0083      0.001      9.104      0.000       0.007       0.010
revol_util             -0.0002   4.86e-05     -4.002      0.000      -0.000   -9.93e-05
total_acc               0.0006      0.000      5.986      0.000       0.000       0.001
int_rate               -0.0077      0.001    -11.109      0.000      -0.009      -0.006
A                      -0.5480      0.041    -13.361      0.000      -0.628      -0.468
B                      -0.5542      0.041    -13.574      0.000      -0.634      -0.474
C                      -0.5618      0.041    -13.608      0.000      -0.643      -0.481
D                      -0.5660      0.042    -13.452      0.000      -0.648      -0.484
E                      -0.5586      0.043    -12.973      0.000      -0.643      -0.474
F                      -0.5585      0.045    -12.540      0.000      -0.646      -0.471
G                      -0.5818      0.054    -10.766      0.000      -0.688      -0.476
=======================================================================================
"""

In [58]:
logit = sm.Probit(y, X3.astype(float)).fit()
logit_me = logit.get_margeff()
logit_me.summary()

         Current function value: 0.356124
         Iterations: 35


<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                 target
Method:                          dydx
At:                           overall
=======================================================================================
                         dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
loan_amnt           -1.691e-06   1.34e-07    -12.576      0.000   -1.95e-06   -1.43e-06
ln_annual_inc           0.0567      0.002     27.783      0.000       0.053       0.061
dti                    -0.0012      0.000     -9.386      0.000      -0.001      -0.001
fico_range_high         0.0004   4.57e-05      8.109      0.000       0.000       0.000
delinq_2yrs             0.0078      0.004      2.193      0.028       0.001       0.015
num_delinq_2yrs        -0.0010      0.002     -0.554      0.580      -0.005       0.003
ln_earliest_cr_line    -0.0040      0.002     -2.128      0.033      -0.008      -0.000
inq_last_6mths          0.0022      0.003      0.849      0.396      -0.003       0.007
num_inq_last_6mths     -0.0121      0.001     -9.730      0.000      -0.015      -0.010
ln_open_acc            -0.0210      0.003     -7.958      0.000      -0.026      -0.016
pub_rec                 0.0090      0.005      1.796      0.073      -0.001       0.019
num_pub_rec             0.0015      0.004      0.416      0.677      -0.006       0.009
ln_revol_bal            0.0080      0.001      8.650      0.000       0.006       0.010
revol_util             -0.0002   4.87e-05     -3.603      0.000      -0.000      -8e-05
total_acc               0.0006      0.000      6.062      0.000       0.000       0.001
int_rate               -0.0038      0.001     -3.212      0.001      -0.006      -0.001
A1                     -0.4872      0.043    -11.301      0.000      -0.572      -0.403
A2                     -0.5000      0.042    -11.791      0.000      -0.583      -0.417
A3                     -0.5121      0.042    -12.150      0.000      -0.595      -0.429
A4                     -0.5264      0.042    -12.601      0.000      -0.608      -0.444
A5                     -0.5329      0.042    -12.773      0.000      -0.615      -0.451
B1                     -0.5364      0.042    -12.835      0.000      -0.618      -0.454
B2                     -0.5375      0.042    -12.831      0.000      -0.620      -0.455
B3                     -0.5416      0.042    -12.884      0.000      -0.624      -0.459
B4                     -0.5478      0.042    -12.963      0.000      -0.631      -0.465
B5                     -0.5552      0.042    -13.071      0.000      -0.638      -0.472
C1                     -0.5570      0.043    -13.055      0.000      -0.641      -0.473
C2                     -0.5607      0.043    -13.050      0.000      -0.645      -0.477
C3                     -0.5726      0.043    -13.249      0.000      -0.657      -0.488
C4                     -0.5753      0.043    -13.240      0.000      -0.660      -0.490
C5                     -0.5687      0.044    -12.963      0.000      -0.655      -0.483
D1                     -0.5834      0.044    -13.215      0.000      -0.670      -0.497
D2                     -0.5815      0.045    -13.058      0.000      -0.669      -0.494
D3                     -0.5813      0.045    -12.961      0.000      -0.669      -0.493
D4                     -0.5890      0.045    -13.051      0.000      -0.677      -0.501
D5                     -0.5763      0.046    -12.629      0.000      -0.666      -0.487
E1                     -0.5808      0.046    -12.592      0.000      -0.671      -0.490
E2                     -0.5903      0.047    -12.656      0.000      -0.682      -0.499
E3                     -0.5795      0.047    -12.269      0.000      -0.672      -0.487
E4                     -0.5876      0.048    -12.262      0.000   

In [72]:
sum_stats = train.groupby(['grade'])[['int_rate', 'annual_inc', 'fico_range_high', 'dti', 'loan_amnt', 'target']].mean()
sum_stats

,int_rate,annual_inc,fico_range_high,dti,loan_amnt,target
grade,,,,,,
A,7.533552,78061.632801,742.191457,14.213065,12528.419975,0.943643
B,11.784927,68131.638415,701.306904,16.463607,12044.910911,0.894520
C,15.175030,65099.059213,686.804995,17.200045,11744.074976,0.842224
D,18.259644,64625.170046,680.909519,17.079463,11350.114301,0.797595
E,20.967960,69862.437261,678.605408,16.816023,12137.651132,0.774016
F,23.139601,69501.044426,678.238210,15.984619,11426.995163,0.740024
G,23.226515,121153.817576,673.090909,17.180000,22742.045455,0.727273


grade
A    1.014733
B    0.999938
C    0.970032
D    0.943233
E    0.936312
F    0.911263
G    0.896193
dtype: float64